*MODEL* KE-2 : 

DATASET MENTAH -> TEXT PRE-PROCESSING (CASE FOLDING - TOKENIZING - FILTERING / STOPWORD REMOVAL - NOMALIZATION) -> EKTRASI FITUR (Sentimen Leksikon Indonesia) -> MAXIMUM ENTROPY

**TRAIN DATASET**

In [ ]:
# Setup (Instalasi Library NLTK)
import nltk
# =======================
import pandas as pd
import numpy as np
training_nonlable = pd.read_csv('trainset.csv')
training_nonlable

,time,id,user,tweet
0,2021-07-27 14:47:14,1420033042303827972,Irsan Aklima,Selain Kasus Covid turun PPKM juga bikin waktu...
1,2021-07-27 14:44:57,1420032466556006406,June,#terimakasihkamu Berkat PPKM Zona merah berkurang
2,2021-07-27 14:33:01,1420029462943502339,Pinka WizArt 💋,lamaan dah PPKM kaga papa.. dedek di rumah jad...
3,2021-07-27 14:29:39,1420028614934999053,Nama Tidak Boleh Kosong,#terimakasihkamu berkat kamu PPKM di kota tang...
4,2021-07-27 14:27:16,1420028014876930067,𝟮𝗻𝗱 𝗛𝗮𝗹𝗳 📍,PPKM jadikan kita lebih dekat dg keluarga #ter...
...,...,...,...,...
1556,2021-07-25 6:33:23,1419183983275888640,imeel🦄,Abis isoman 2 minggu terus PPKM sakit banget a...
1557,2021-07-25 0:55:26,1419098935126224901,your name here,bulan ini ngerasa kosong banget anjir baru nge...
1558,2021-07-25 13:05:30,1419282662200516613,spam⁷,Capek bgt PPKM diperpanjang lagi. Mental udah ...
1559,2021-07-25 12:31:33,1419274117455368195,7:3 🍀,Sebenernya PPKM selama ini gak akan efektif ka...


**TEST DATASET**

In [ ]:
testset_nonlable = pd.read_csv('testset.csv')
testset_nonlable

,time,id,user,tweet
0,2021-07-23 21:26:31,1418683972465225730,dooo,Ppkm mempersulit hayalankuu..Wahai covid anda ...
1,2021-07-23 15:59:34,1418601693466529798,maintire,Darurat PPKM membuat perekonomian nasional di ...
2,2021-07-23 11:49:36,1418538788096135174,ALVENA,PPKM yang sekarang lebih berat
3,2021-07-25 16:03:19,1419327411234119689,sheba,Ppkm diperpanjang jadwal berantakan krn mundur...
4,2021-07-25 15:27:53,1419318492973527044,iqro,terus aja perpanjang ppkm nya terus aja makin ...
...,...,...,...,...
517,2021-07-25 12:29:16,1419273544693780480,Éoheort sons of Dernbrand,ppkm ternyata bermanfaat juga ya menjauhkan se...
518,2021-07-25 10:15:44,1419239941440118786,Kenalin aku..,Terima kasih PPKM karena sudah mengajarkanku u...
519,2021-07-25 3:20:53,1419135540121669639,Retno DrW Skincare Banyuwangi,jazakumulallah khoiran .....Hoby yang di bayar...
520,2021-07-27 14:49:49,1420033689107509258,Irsan Aklima,Sudah banyak manfaat pemberlakuan PPKM #terima...


# **TEXT PRE-PROCESSING**

**CASE FOLDING**

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
training_nonlable['tweet'] = training_nonlable['tweet'].str.lower()
# ---------------------------------------------------
testset_nonlable['tweet'] = testset_nonlable['tweet'].str.lower()


print('Case Folding Result : \n')
print(training_nonlable['tweet'])
print('\n\n\n')
print(testset_nonlable['tweet'])

Case Folding Result : 

0       selain kasus covid turun ppkm juga bikin waktu...
1       #terimakasihkamu berkat ppkm zona merah berkurang
2       lamaan dah ppkm kaga papa.. dedek di rumah jad...
3       #terimakasihkamu berkat kamu ppkm di kota tang...
4       ppkm jadikan kita lebih dekat dg keluarga #ter...
                              ...                        
1556    abis isoman 2 minggu terus ppkm sakit banget a...
1557    bulan ini ngerasa kosong banget anjir baru nge...
1558    capek bgt ppkm diperpanjang lagi. mental udah ...
1559    sebenernya ppkm selama ini gak akan efektif ka...
1560    hari ini lagi sedih capek bgt stress karena ba...
Name: tweet, Length: 1561, dtype: object




0      ppkm mempersulit hayalankuu..wahai covid anda ...
1      darurat ppkm membuat perekonomian nasional di ...
2                         ppkm yang sekarang lebih berat
3      ppkm diperpanjang jadwal berantakan krn mundur...
4      terus aja perpanjang ppkm nya terus aja makin ...
        

**TOKENIZING**

In [ ]:
nltk.download('punkt')

import string 
import re #regex library


# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist


# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
training_nonlable['tweet'] = training_nonlable['tweet'].apply(remove_tweet_special)
# ---------------------------------------------------
testset_nonlable['tweet'] = testset_nonlable['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

training_nonlable['tweet'] = training_nonlable['tweet'].apply(remove_number)
# ---------------------------------------------------
testset_nonlable['tweet'] = testset_nonlable['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

training_nonlable['tweet'] = training_nonlable['tweet'].apply(remove_punctuation)
# ---------------------------------------------------
testset_nonlable['tweet'] = testset_nonlable['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

training_nonlable['tweet'] = training_nonlable['tweet'].apply(remove_whitespace_LT)
# ---------------------------------------------------
testset_nonlable['tweet'] = testset_nonlable['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

training_nonlable['tweet'] = training_nonlable['tweet'].apply(remove_whitespace_multiple)
# ---------------------------------------------------
testset_nonlable['tweet'] = testset_nonlable['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

training_nonlable['tweet'] = training_nonlable['tweet'].apply(remove_singl_char)
# ---------------------------------------------------
testset_nonlable['tweet'] = testset_nonlable['tweet'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

training_nonlable['tweet_tokens'] = training_nonlable['tweet'].apply(word_tokenize_wrapper)
# ---------------------------------------------------
testset_nonlable['tweet_tokens'] = testset_nonlable['tweet'].apply(word_tokenize_wrapper)

print(training_nonlable['tweet_tokens'])
print('\n\n\n')
print(testset_nonlable['tweet_tokens'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
0       [selain, kasus, covid, turun, ppkm, juga, biki...
1                  [berkat, ppkm, zona, merah, berkurang]
2       [lamaan, dah, ppkm, kaga, papa, dedek, di, rum...
3       [berkat, kamu, ppkm, di, kota, tangerang, menj...
4       [ppkm, jadikan, kita, lebih, dekat, dg, keluarga]
                              ...                        
1556    [abis, isoman, minggu, terus, ppkm, sakit, ban...
1557    [bulan, ini, ngerasa, kosong, banget, anjir, b...
1558    [capek, bgt, ppkm, diperpanjang, lagi, mental,...
1559    [sebenernya, ppkm, selama, ini, gak, akan, efe...
1560    [hari, ini, lagi, sedih, capek, bgt, stress, k...
Name: tweet_tokens, Length: 1561, dtype: object




0      [ppkm, mempersulit, hayalankuuwahai, covid, an...
1      [darurat, ppkm, membuat, perekonomian, nasiona...
2                   [ppkm, yang, sekarang, lebih, berat]
3      [ppkm, diperpanjang, jadwa

**NORMALIZATION\FORMALIZATION**

In [ ]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

training_nonlable['tweet_normalized'] = training_nonlable['tweet_tokens'].apply(normalized_term)
# ---------------------------------------------------
testset_nonlable['tweet_normalized'] = testset_nonlable['tweet_tokens'].apply(normalized_term)

print(training_nonlable['tweet_normalized'])
print('\n\n\n')
print(testset_nonlable['tweet_normalized'])

0       [selain, kasus, covid, turun, ppkm, juga, biki...
1                  [berkat, ppkm, zona, merah, berkurang]
2       [lamaan, dah, ppkm, kaga, papa, dedek, di, rum...
3       [berkat, kamu, ppkm, di, kota, tangerang, menj...
4       [ppkm, jadikan, kita, lebih, dekat, dg, keluarga]
                              ...                        
1556    [habis, isoman, minggu, terus, ppkm, sakit, ba...
1557    [bulan, ini, ngerasa, kosong, banget, anjir, b...
1558    [capek, bgt, ppkm, diperpanjang, lagi, mental,...
1559    [sebenernya, ppkm, selama, ini, tidak, akan, e...
1560    [hari, ini, lagi, sedih, capek, bgt, stress, k...
Name: tweet_normalized, Length: 1561, dtype: object




0      [ppkm, kesulitan, hayalankuuwahai, covid, anda...
1      [darurat, ppkm, membuat, perekonomian, nasiona...
2                   [ppkm, yang, sekarang, lebih, berat]
3      [ppkm, diperpanjang, jadwal, berantakan, krn, ...
4      [terus, aja, perpanjang, ppkm, nya, terus, aja...
                     

**FILTERING (Stopword Removal) and (Removing 'PPKM' & 'DARURAT')**

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword (MENGHAPUS 'PPKM' & 'DARURAT')
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'ppkm', 'darurat', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'birthday', 'doang', 'dong', 'cari', 'itu',
                       'krn', 'karna', 'nya', 'nih', 'sih', 'udh', 'sampai', 'kenapa', 'duluan', 'ada', 'abis', 'ugh', 'pengen',
                       'si', 'tau', 'tuh', 'utk', 'ya', 'trs', 'sm', 'padahal', 'lagi', 'dpt', 'dapat', 'dapet', 'ken', 'mlu',
                       'jd', 'sdh', 'aja', 'n', 't', 'pas', 'psbb', 'yang', 'apa', 'yang', 'banyak', 'buat', 'pls', 'mulu', 'cari',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt', 'emg', 'buat', 'sdg', 'pada', 'pda', 'allah', 'ydh', 'yaudah',
                       '&amp', 'banget', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)

# tidak menghilangkan kata negasi
for word in list_stopwords.copy():
    if (word == 'tidak'):
      list_stopwords.remove('tidak')
    elif (word == 'kagak'):
      list_stopwords.remove('kagak')
    elif (word == 'tak'):
      list_stopwords.remove('tak')
    elif (word == 'enggak'):
      list_stopwords.remove('enggak')
    elif (word == 'bukan'):
      list_stopwords.remove('bukan')
    elif (word == 'masalah'):
      list_stopwords.remove('masalah')
    elif (word == 'guna'):
      list_stopwords.remove('guna')

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

training_nonlable['tweet_tokens_WSW'] = training_nonlable['tweet_normalized'].apply(stopwords_removal)
# ---------------------------------------------------
testset_nonlable['tweet_tokens_WSW'] = testset_nonlable['tweet_normalized'].apply(stopwords_removal)  

print(training_nonlable['tweet_tokens_WSW'])
print('\n\n\n')
print(testset_nonlable['tweet_tokens_WSW'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
0                                [covid, turun, keluarga]
1                        [berkat, zona, merah, berkurang]
2           [lamaan, kaga, papa, dedek, rumah, produktif]
3                       [berkat, kota, tangerang, tertib]
4                                     [jadikan, keluarga]
                              ...                        
1556    [habis, isoman, minggu, sakit, asli, tidak, ng...
1557    [ngerasa, kosong, anjir, ngeh, tgl, diterapinn...
1558    [capek, diperpanjang, mental, tidak, kerjaan, ...
1559    [sebenernya, tidak, efektif, tidak, jamin, rak...
1560    [sedih, capek, stress, tugas, kuliah, kebingun...
Name: tweet_tokens_WSW, Length: 1561, dtype: object




0                    [kesulitan, hayalankuuwahai, covid]
1                 [perekonomian, nasional, ri, terpuruk]
2                                                [berat]
3             [diperpanj

In [ ]:
# Save hasil text prepocessing

training_nonlable['tweet_tokens_WSW'].to_csv("training_nonlable.csv")
testset_nonlable['tweet_tokens_WSW'].to_csv("testset_nonlable.csv")

**EKTRASI FITUR (SENTIMEN LEKSIKON INDONESIA)**

In [ ]:
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

     |████████████████████████████████| 209 kB 4.0 MB/s 


In [ ]:
negasi = ['bukan','tidak','kagak','nggak', 'bosen', 'capek', 'mati', 'astagfirullah', 'stress', 'emosi', 'kasar', 'suntuk', 'gila', 'hancur', 'rusak', 
          'miskin', 'anjir', 'susah', 'sedih', 'tutup', 'sepi', 'lelah', 'anjing', 'sengsara']
lesikon_indonesia = pd.read_csv('sentimen leksikon Indonesia.csv')
len(lesikon_indonesia)

10191

In [ ]:
lesikon_indonesia

,word,weight
0,adika,5
1,agam,5
2,alhamdulillah,5
3,allah,5
4,amanah,5
...,...,...
10186,wajar,-5
10187,walang hati,-5
10188,yang,-5
10189,yaoi,-5


In [ ]:
lesikon_indonesia_word = lesikon_indonesia['word'].to_list()
len(lesikon_indonesia_word)

10191

In [ ]:
training_data = pd.read_csv('training_nonlable.csv')
del training_data['Unnamed: 0']
training_data

,tweet_tokens_WSW
0,"['covid', 'turun', 'keluarga']"
1,"['berkat', 'zona', 'merah', 'berkurang']"
2,"['lamaan', 'kaga', 'papa', 'dedek', 'rumah', '..."
3,"['berkat', 'kota', 'tangerang', 'tertib']"
4,"['jadikan', 'keluarga']"
...,...
1556,"['habis', 'isoman', 'minggu', 'sakit', 'asli',..."
1557,"['ngerasa', 'kosong', 'anjir', 'ngeh', 'tgl', ..."
1558,"['capek', 'diperpanjang', 'mental', 'tidak', '..."
1559,"['sebenernya', 'tidak', 'efektif', 'tidak', 'j..."


In [ ]:
testing_data = pd.read_csv('testset_nonlable.csv')
del testing_data['Unnamed: 0']
testing_data

,tweet_tokens_WSW
0,"['kesulitan', 'hayalankuuwahai', 'covid']"
1,"['perekonomian', 'nasional', 'ri', 'terpuruk']"
2,['berat']
3,"['diperpanjang', 'jadwal', 'berantakan', 'mund..."
4,"['perpanjang', 'penganguran']"
...,...
517,"['bermanfaat', 'menjauhkan', 'sendernya', 'kem..."
518,"['terima', 'kasih', 'mengajarkanku', 'setia', ..."
519,"['jazakumulallah', 'khoiran', 'hoby', 'bayar',..."
520,"['manfaat', 'pemberlakuan']"


**PERHITUNGAN SCORE SENTIMEN TRAINING**

In [ ]:
sencol =[]
senrow =np.array([])
nsen = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()
sentiment_list = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #if not, than add new word
        sencol.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lesikon_indonesia['weight'][lesikon_indonesia_word.index(word)]
    else:
        sentiment += lesikon_indonesia['weight'][lesikon_indonesia_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in range(len(training_data)):
    nsen = senrow.shape[0]
    words = word_tokenize(training_data['tweet_tokens_WSW'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lesikon_indonesia_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lesikon_indonesia_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lesikon_indonesia_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lesikon_indonesia_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)

In [ ]:
sencol.append('score')
sentiment_array = np.array(sentiment_list).reshape(senrow.shape[0],1)
sentiment_data = np.hstack((senrow,sentiment_array))
df_sen = pd.DataFrame(sentiment_data,columns = sencol)

# Save dataframe yang berisi sekantong kata dan segmen yang telah dihitung sebelumnya
df_sen.to_csv("hasil_scoring.csv")

df_sen.head(10)

,turun,keluarga,berkat,merah,kurang,lama,rumah,produktif,kota,tertib,jadi,mantap,tahan,patuh,dampak,guna,alhamdulillah,efektif,hasil,lewat,erti,arti,bagi,bekas,terima,kasih,mati,positif,limpah,rezeki,makan,bantu,bukti,tidak,indah,upaya,tangan,pandemi,rajin,pagi,...,luluh,hajar,kiamat,santai,domba,pelihara,tunduk,konsekuensi,sidang,canda,antem,pola,surut,amuk,idiot,tendang,kuasa,dosa,tetes,mata,konyol,begundal,ngadepin,dewasa,banting,miris,kekang,bengkak,agama,didik,apas,mulus,die,masuk,ngomongin,bakar,gede,nyalahin,ngerjain,score
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
5,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
6,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5


**PERHITUNGAN SCORE SENTIMEN TESTING**

In [ ]:
sencol_testing =[]
senrow_testing =np.array([])
nsen_testing = 0
factory_testing = StemmerFactory()
stemmer_testing = factory.create_stemmer()
sentiment_list_testing = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol_testing,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol_testing:
        sen[sencol_testing.index(word)] += 1
    else:
    #if not, than add new word
        sencol_testing.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lesikon_indonesia['weight'][lesikon_indonesia_word.index(word)]
    else:
        sentiment += lesikon_indonesia['weight'][lesikon_indonesia_word.index(word)]
    
    return sen,sencol_testing,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in range(len(testing_data)):
    nsen_testing = senrow_testing.shape[0]
    words = word_tokenize(testing_data['tweet_tokens_WSW'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol_testing)>0:
        sen =[0 for j in range(len(sencol_testing))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lesikon_indonesia_word :
            sen,sencol_testing,sentiment,add= found_word(ind,words,word,sen,sencol_testing,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer_testing.stem(word)
            if kata_dasar in lesikon_indonesia_word:
                sen,sencol_testing,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol_testing,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lesikon_indonesia_word):
                        sen,sencol_testing,sentiment,add= found_word(ind,words,back_1,sen,sencol_testing,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lesikon_indonesia_word:
                            sen,sencol_testing,sentiment,add= found_word(ind,words,back_2,sen,sencol_testing,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen_testing==0):
                senrow_testing = np.zeros([i,add],dtype=int)
            elif(i!=nsen_testing):
                padding_h = np.zeros([nsen_testing,add],dtype=int)
                senrow_testing = np.hstack((senrow_testing,padding_h))
                padding_v = np.zeros([(i-nsen_testing),senrow_testing.shape[1]],dtype=int)
                senrow_testing = np.vstack((senrow_testing,padding_v))
            else:
                padding =np.zeros([nsen_testing,add],dtype=int)
                senrow_testing = np.hstack((senrow_testing,padding))
            senrow_testing = np.vstack((senrow_testing,sen))
        if i==0:
            senrow_testing = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen_testing>0):
        senrow_testing = np.vstack((senrow_testing,sen))
        
    sentiment_list_testing.append(sentiment)

In [ ]:
sencol_testing.append('score')
sentiment_array_testing = np.array(sentiment_list_testing).reshape(senrow_testing.shape[0],1)
sentiment_data_testing = np.hstack((senrow_testing,sentiment_array_testing))
df_sen_testing = pd.DataFrame(sentiment_data_testing,columns = sencol_testing)

# Save dataframe yang berisi sekantong kata dan segmen yang telah dihitung sebelumnya
df_sen_testing.to_csv("hasil_scoring_testing.csv")

df_sen_testing.head(10)

,panjang,tengah,kerja,hotel,perut,tidak,hasil,mantap,jual,arif,bentuk,cair,tugas,lapang,alhamdulillah,presiden,baik,gua,nyari,hati,tutup,pasang,ldr,tambah,batas,tinggal,sepi,serah,beli,akses,jerit,makan,nganggur,gejolak,sumpah,sekolah,tingkat,dampak,cari,susah,...,nyaman,kreatif,kreativitas,kreasi,aamiin,makmur,partisipasi,lumayan,positif,pohon,lahir,aparat,jabar,ajak,lupa,lancar,sadar,ucap,hijau,ketat,kawal,juang,rebah,manusia,nunggu,selesai,syukur,senang,air,yaudahlah,buka,manfaat,jauh,terima,kasih,setia,kasur,bayar,berkah,score
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
5,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


**HASIL PENGHITUNGAN SCORE TRAINING**

In [ ]:
cek_df = pd.DataFrame([])
cek_df['original_text'] = training_nonlable['tweet'].copy()
cek_df['clean_text'] = training_data['tweet_tokens_WSW'].copy()
cek_df['score']  = df_sen['score'].copy()
#cek_df['sentimen']  = df_sen_pos['sentiment'].copy()

# Save dataframe hasil scoring
cek_df.to_csv("training_scoring.csv")

cek_df

,original_text,clean_text,score
0,selain kasus covid turun ppkm juga bikin waktu...,"['covid', 'turun', 'keluarga']",5
1,berkat ppkm zona merah berkurang,"['berkat', 'zona', 'merah', 'berkurang']",2
2,lamaan dah ppkm kaga papa dedek di rumah jadi ...,"['lamaan', 'kaga', 'papa', 'dedek', 'rumah', '...",1
3,berkat kamu ppkm di kota tangerang menjadi tertib,"['berkat', 'kota', 'tangerang', 'tertib']",4
4,ppkm jadikan kita lebih dekat dg keluarga,"['jadikan', 'keluarga']",5
...,...,...,...
1556,abis isoman minggu terus ppkm sakit banget asl...,"['habis', 'isoman', 'minggu', 'sakit', 'asli',...",-2
1557,bulan ini ngerasa kosong banget anjir baru nge...,"['ngerasa', 'kosong', 'anjir', 'ngeh', 'tgl', ...",-28
1558,capek bgt ppkm diperpanjang lagi mental udah m...,"['capek', 'diperpanjang', 'mental', 'tidak', '...",-28
1559,sebenernya ppkm selama ini gak akan efektif ka...,"['sebenernya', 'tidak', 'efektif', 'tidak', 'j...",-16


In [ ]:
def labeling(cek_df):
    temp = []
    for score in cek_df:
        if score>0:
            temp.append('positif')
        else:
            temp.append('negatif')
    return temp

cek_df['sentiment'] = labeling(cek_df['score'])
print(cek_df['sentiment'])

0       positif
1       positif
2       positif
3       positif
4       positif
         ...   
1556    negatif
1557    negatif
1558    negatif
1559    negatif
1560    negatif
Name: sentiment, Length: 1561, dtype: object


In [ ]:
df_labeled = pd.DataFrame([])
df_labeled['original_text'] = training_nonlable['tweet'].copy()
df_labeled['clean_text'] = training_nonlable['tweet_tokens_WSW'].copy()
df_labeled['score']  = df_sen['score'].copy()
df_labeled['sentiment']  = cek_df['sentiment'].copy()

# Save dataframe hasil scoring
df_labeled.to_csv("training_labled.csv")

df_labeled

,original_text,clean_text,score,sentiment
0,selain kasus covid turun ppkm juga bikin waktu...,"[covid, turun, keluarga]",5,positif
1,berkat ppkm zona merah berkurang,"[berkat, zona, merah, berkurang]",2,positif
2,lamaan dah ppkm kaga papa dedek di rumah jadi ...,"[lamaan, kaga, papa, dedek, rumah, produktif]",1,positif
3,berkat kamu ppkm di kota tangerang menjadi tertib,"[berkat, kota, tangerang, tertib]",4,positif
4,ppkm jadikan kita lebih dekat dg keluarga,"[jadikan, keluarga]",5,positif
...,...,...,...,...
1556,abis isoman minggu terus ppkm sakit banget asl...,"[habis, isoman, minggu, sakit, asli, tidak, ng...",-2,negatif
1557,bulan ini ngerasa kosong banget anjir baru nge...,"[ngerasa, kosong, anjir, ngeh, tgl, diterapinn...",-28,negatif
1558,capek bgt ppkm diperpanjang lagi mental udah m...,"[capek, diperpanjang, mental, tidak, kerjaan, ...",-28,negatif
1559,sebenernya ppkm selama ini gak akan efektif ka...,"[sebenernya, tidak, efektif, tidak, jamin, rak...",-16,negatif


**HASIL PENGHITUNGAN SCORE TESTING**

In [ ]:
cek_df_testing = pd.DataFrame([])
cek_df_testing['original_text'] = testset_nonlable['tweet'].copy()
cek_df_testing['clean_text'] = testing_data['tweet_tokens_WSW'].copy()
cek_df_testing['score']  = df_sen_testing['score'].copy()
#cek_df['sentimen']  = df_sen_pos['sentiment'].copy()

# Save dataframe hasil scoring
cek_df_testing.to_csv("testing_scoring.csv")

cek_df_testing

,original_text,clean_text,score
0,ppkm mempersulit hayalankuuwahai covid anda lu...,"['kesulitan', 'hayalankuuwahai', 'covid']",0
1,darurat ppkm membuat perekonomian nasional di ...,"['perekonomian', 'nasional', 'ri', 'terpuruk']",0
2,ppkm yang sekarang lebih berat,['berat'],0
3,ppkm diperpanjang jadwal berantakan krn mundur...,"['diperpanjang', 'jadwal', 'berantakan', 'mund...",2
4,terus aja perpanjang ppkm nya terus aja makin ...,"['perpanjang', 'penganguran']",2
...,...,...,...
517,ppkm ternyata bermanfaat juga ya menjauhkan se...,"['bermanfaat', 'menjauhkan', 'sendernya', 'kem...",1
518,terima kasih ppkm karena sudah mengajarkanku u...,"['terima', 'kasih', 'mengajarkanku', 'setia', ...",-7
519,jazakumulallah khoiran hoby yang di bayar berk...,"['jazakumulallah', 'khoiran', 'hoby', 'bayar',...",5
520,sudah banyak manfaat pemberlakuan ppkm,"['manfaat', 'pemberlakuan']",5


In [ ]:
def labeling(cek_df_testing):
    temp = []
    for score in cek_df_testing:
        if score>0:
            temp.append('positif')
        else:
            temp.append('negatif')
    return temp

cek_df_testing['sentiment'] = labeling(cek_df_testing['score'])
print(cek_df_testing['sentiment'])

0      negatif
1      negatif
2      negatif
3      positif
4      positif
        ...   
517    positif
518    negatif
519    positif
520    positif
521    positif
Name: sentiment, Length: 522, dtype: object


In [ ]:
df_labeled_testing = pd.DataFrame([])
df_labeled_testing['original_text'] = testset_nonlable['tweet'].copy()
df_labeled_testing['clean_text'] = testset_nonlable['tweet_tokens_WSW'].copy()
df_labeled_testing['score']  = df_sen_testing['score'].copy()
df_labeled_testing['sentiment']  = cek_df_testing['sentiment'].copy()

# Save dataframe hasil scoring
df_labeled_testing.to_csv("testing_labled.csv")

df_labeled_testing

,original_text,clean_text,score,sentiment
0,ppkm mempersulit hayalankuuwahai covid anda lu...,"[kesulitan, hayalankuuwahai, covid]",0,negatif
1,darurat ppkm membuat perekonomian nasional di ...,"[perekonomian, nasional, ri, terpuruk]",0,negatif
2,ppkm yang sekarang lebih berat,[berat],0,negatif
3,ppkm diperpanjang jadwal berantakan krn mundur...,"[diperpanjang, jadwal, berantakan, mundur]",2,positif
4,terus aja perpanjang ppkm nya terus aja makin ...,"[perpanjang, penganguran]",2,positif
...,...,...,...,...
517,ppkm ternyata bermanfaat juga ya menjauhkan se...,"[bermanfaat, menjauhkan, sendernya, kemaksiatan]",1,positif
518,terima kasih ppkm karena sudah mengajarkanku u...,"[terima, kasih, mengajarkanku, setia, setia, k...",-7,negatif
519,jazakumulallah khoiran hoby yang di bayar berk...,"[jazakumulallah, khoiran, hoby, bayar, berkah]",5,positif
520,sudah banyak manfaat pemberlakuan ppkm,"[manfaat, pemberlakuan]",5,positif


# **TEXT CLASSIFICATION MENGGUNAKAN METODE MAXIMUM ENTROPY**

**SEPARATE TRAIN DATASET BASED ON SENTIMEN LABEL**

In [ ]:
trainset_positif = df_labeled[df_labeled['sentiment'] == "positif"]

# Save dataframe yang berlebel positif
trainset_positif.to_csv("trainset_positif.csv")

positive_trainset = pd.read_csv('trainset_positif.csv')
del positive_trainset['Unnamed: 0']
positive_trainset

,original_text,clean_text,score,sentiment
0,selain kasus covid turun ppkm juga bikin waktu...,"['covid', 'turun', 'keluarga']",5,positif
1,berkat ppkm zona merah berkurang,"['berkat', 'zona', 'merah', 'berkurang']",2,positif
2,lamaan dah ppkm kaga papa dedek di rumah jadi ...,"['lamaan', 'kaga', 'papa', 'dedek', 'rumah', '...",1,positif
3,berkat kamu ppkm di kota tangerang menjadi tertib,"['berkat', 'kota', 'tangerang', 'tertib']",4,positif
4,ppkm jadikan kita lebih dekat dg keluarga,"['jadikan', 'keluarga']",5,positif
...,...,...,...,...
806,sekeluarga gak ada penghasilan aku mau kerja t...,"['sekeluarga', 'tidak', 'penghasilan', 'diperp...",7,positif
807,assalamualaikum pak demi allah saya sudah tida...,"['assalamualaikum', 'tidak', 'sanggup', 'diper...",1,positif
808,sdh gak bisa nanggung hidup rakyatnya malah bi...,"['tidak', 'nanggung', 'hidup', 'rakyatnya', 'h...",1,positif
809,capek ngadepin kakak yang gak dewasa ayah ibuk...,"['capek', 'ngadepin', 'kakak', 'tidak', 'dewas...",7,positif


In [ ]:
trainset_negatif = df_labeled[df_labeled['sentiment'] == "negatif"]

# Save dataframe yang berlebel positif
trainset_negatif.to_csv("trainset_negatif.csv")

negative_trainset = pd.read_csv('trainset_negatif.csv')
del negative_trainset['Unnamed: 0']
negative_trainset

,original_text,clean_text,score,sentiment
0,ppkm memberikan dampak baik dan ada gunanya,"['dampak', 'guna']",-2,negatif
1,selain itu dampak yang sangat baik untuk jawa ...,"['dampak', 'jawa', 'barat', 'kematiannya', 'me...",-7,negatif
2,ppkm tidak siasia alhamdulillah,"['tidak', 'siasia', 'alhamdulillah']",0,negatif
3,berbagai upaya penanganan yang dijalankan peme...,"['upaya', 'penanganan', 'dijalankan', 'pemerin...",0,negatif
4,disiplin terhadap kebijakan ppkm darurat sudah...,"['disiplin', 'kebijakan', 'atasi', 'pandemi', ...",-3,negatif
...,...,...,...,...
745,abis isoman minggu terus ppkm sakit banget asl...,"['habis', 'isoman', 'minggu', 'sakit', 'asli',...",-2,negatif
746,bulan ini ngerasa kosong banget anjir baru nge...,"['ngerasa', 'kosong', 'anjir', 'ngeh', 'tgl', ...",-28,negatif
747,capek bgt ppkm diperpanjang lagi mental udah m...,"['capek', 'diperpanjang', 'mental', 'tidak', '...",-28,negatif
748,sebenernya ppkm selama ini gak akan efektif ka...,"['sebenernya', 'tidak', 'efektif', 'tidak', 'j...",-16,negatif


In [ ]:
testset_positif = df_labeled_testing[df_labeled_testing['sentiment'] == "positif"]

# Save dataframe yang berlebel positif
testset_positif.to_csv("testset_positif.csv")

positive_testset = pd.read_csv('testset_positif.csv')
del positive_testset['Unnamed: 0']
positive_testset

,original_text,clean_text,score,sentiment
0,ppkm diperpanjang jadwal berantakan krn mundur...,"['diperpanjang', 'jadwal', 'berantakan', 'mund...",2,positif
1,terus aja perpanjang ppkm nya terus aja makin ...,"['perpanjang', 'penganguran']",2,positif
2,tentunya hal ini perpanjangan ppkm makin membu...,"['perpanjangan', 'rakyat', 'menengah', 'kebawa...",4,positif
3,ppkm di perpanjang mau gerak dalam keadaan urg...,"['perpanjang', 'gerak', 'urgent', 'kesulitan']",2,positif
4,ppkm di perpanjang tapi nasib pekerja hotel ma...,"['perpanjang', 'nasib', 'pekerja', 'hotel', 'h...",5,positif
...,...,...,...,...
242,mau ppkm diperpanjang lg atau gmn yaudalah iku...,"['diperpanjang', 'yaudalah', 'ikutin', 'pemeri...",5,positif
243,ppkm ternyata bermanfaat juga ya menjauhkan se...,"['bermanfaat', 'menjauhkan', 'sendernya', 'kem...",1,positif
244,jazakumulallah khoiran hoby yang di bayar berk...,"['jazakumulallah', 'khoiran', 'hoby', 'bayar',...",5,positif
245,sudah banyak manfaat pemberlakuan ppkm,"['manfaat', 'pemberlakuan']",5,positif


In [ ]:
testset_negatif = df_labeled_testing[df_labeled_testing['sentiment'] == "negatif"]

# Save dataframe yang berlebel positif
testset_negatif.to_csv("testset_negatif.csv")

negative_testset = pd.read_csv('testset_negatif.csv')
del negative_testset['Unnamed: 0']
negative_testset

,original_text,clean_text,score,sentiment
0,ppkm mempersulit hayalankuuwahai covid anda lu...,"['kesulitan', 'hayalankuuwahai', 'covid']",0,negatif
1,darurat ppkm membuat perekonomian nasional di ...,"['perekonomian', 'nasional', 'ri', 'terpuruk']",0,negatif
2,ppkm yang sekarang lebih berat,['berat'],0,negatif
3,ppkm level sangat menyulitkan,"['level', 'menyulitkan']",0,negatif
4,day ppkm benar benar membosenkan,"['day', 'membosenkan']",0,negatif
...,...,...,...,...
270,setelah ppkm diterapkan tak ada lagi antrian d...,"['diterapkan', 'tidak', 'antrian', 'rs']",-5,negatif
271,penerapan ppkm berhasil menekan penyebaran dan...,"['penerapan', 'berhasil', 'menekan', 'penyebar...",0,negatif
272,ppkm sangat berhasil tekan penularan pandemi ini,"['berhasil', 'tekan', 'penularan', 'pandemi']",-5,negatif
273,alhamduillah ppkm darurat berhasil menekan lon...,"['alhamduillah', 'berhasil', 'menekan', 'lonja...",-1,negatif


**CLASSIFICATION**

In [ ]:
print(trainset_positif['clean_text'])
print('\n\n\n')
print(trainset_negatif['clean_text'])

0                                [covid, turun, keluarga]
1                        [berkat, zona, merah, berkurang]
2           [lamaan, kaga, papa, dedek, rumah, produktif]
3                       [berkat, kota, tangerang, tertib]
4                                     [jadikan, keluarga]
                              ...                        
1482    [sekeluarga, tidak, penghasilan, diperpanjang,...
1498    [assalamualaikum, tidak, sanggup, diperpanjang...
1541    [tidak, nanggung, hidup, rakyatnya, hidup, rak...
1543    [capek, ngadepin, kakak, tidak, dewasa, ayah, ...
1546    [perpanjang, rakyat, paksa, tinggal, rumah, sb...
Name: clean_text, Length: 811, dtype: object




7                                          [dampak, guna]
14            [dampak, jawa, barat, kematiannya, menurun]
20                         [tidak, siasia, alhamdulillah]
31      [upaya, penanganan, dijalankan, pemerintah, pa...
40      [disiplin, kebijakan, atasi, pandemi, covid, i...
                       

In [ ]:
print(testset_positif['clean_text'])
print('\n\n\n')
print(testset_negatif['clean_text'])

3             [diperpanjang, jadwal, berantakan, mundur]
4                              [perpanjang, penganguran]
5      [perpanjangan, rakyat, menengah, kebawah, kesu...
9                 [perpanjang, gerak, urgent, kesulitan]
10           [perpanjang, nasib, pekerja, hotel, hancur]
                             ...                        
516    [diperpanjang, yaudalah, ikutin, pemerintak, t...
517     [bermanfaat, menjauhkan, sendernya, kemaksiatan]
519       [jazakumulallah, khoiran, hoby, bayar, berkah]
520                              [manfaat, pemberlakuan]
521               [kabarnya, kota, hasil, keren, lonjak]
Name: clean_text, Length: 247, dtype: object




0                    [kesulitan, hayalankuuwahai, covid]
1                 [perekonomian, nasional, ri, terpuruk]
2                                                [berat]
6                                   [level, menyulitkan]
7                                     [day, membosenkan]
                             ...       

In [ ]:
import pandas as pd
import collections
import nltk.classify.util, nltk.metrics
from nltk.classify import MaxentClassifier, SklearnClassifier
import sklearn.metrics as metrics
from sklearn.svm import LinearSVC, SVC
import csv
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import random
import itertools
from nltk.metrics import ConfusionMatrix
from collections import defaultdict

In [ ]:
def word_feats(words):	
	return dict([(word, True) for word in words])

# Calculating Precision, Recall & F-measure
def evaluate_classifier(featx):
	
	positive_trainsett = [(featx(f), 'pos') for f in trainset_positif['clean_text']]
	negative_trainsett = [(featx(f), 'neg') for f in trainset_negatif['clean_text']]
	positive_testsett = [(featx(f), 'pos') for f in testset_positif['clean_text']]
	negative_testsett = [(featx(f), 'neg') for f in testset_negatif['clean_text']]

	trainfeats = negative_trainsett + positive_trainsett
	testfeats = negative_testsett + positive_testsett
	
	classifier_list = ['maxent'] 	
		
	for cl in classifier_list:
		if cl == 'maxent':
			classifierName = 'Maximum Entropy'
			classifier = MaxentClassifier.train(trainfeats, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)
			
		refsets = collections.defaultdict(set)
		testsets = collections.defaultdict(set)
		labels = []
		tests = []
		for i, (feats, label) in enumerate(testfeats):
				refsets[label].add(i)
				observed = classifier.classify(feats)
				testsets[observed].add(i)
				labels.append(label)
				tests.append(observed)
		print(positive_testsett)
		print(negative_testsett)
		print(metrics.confusion_matrix(labels, tests))
		print(nltk.ConfusionMatrix(labels, tests))

		accuracy = nltk.classify.util.accuracy(classifier, testfeats)
		pos_precision = nltk.precision(refsets['pos'], testsets['pos'])
		pos_recall = nltk.recall(refsets['pos'], testsets['pos'])
		pos_fmeasure = nltk.f_measure(refsets['pos'], testsets['pos'])
		neg_precision = nltk.precision(refsets['neg'], testsets['neg'])
		neg_recall = nltk.recall(refsets['neg'], testsets['neg'])
		neg_fmeasure =  nltk.f_measure(refsets['neg'], testsets['neg'])
		
		print ('---------------------------------------')
		print ('SINGLE FOLD RESULT ' + '(' + classifierName + ')')
		print ('---------------------------------------')
		print ('accuracy:', accuracy)
		print ('precision', (pos_precision + neg_precision) / 2)
		print ('recall', (pos_recall + neg_recall) / 2)
		print ('f-measure', (pos_fmeasure + neg_fmeasure) / 2	)
				
		#classifier.show_most_informative_features()
	
		
evaluate_classifier(word_feats)

[({'diperpanjang': True, 'jadwal': True, 'berantakan': True, 'mundur': True}, 'pos'), ({'perpanjang': True, 'penganguran': True}, 'pos'), ({'perpanjangan': True, 'rakyat': True, 'menengah': True, 'kebawah': True, 'kesulitan': True}, 'pos'), ({'perpanjang': True, 'gerak': True, 'urgent': True, 'kesulitan': True}, 'pos'), ({'perpanjang': True, 'nasib': True, 'pekerja': True, 'hotel': True, 'hancur': True}, 'pos'), ({'diperpanjang': True, 'corona': True, 'merajalela': True}, 'pos'), ({'kekecewaan': True, 'perpanjang': True}, 'pos'), ({'diperpanjang': True, 'les': True, 'terbengkalai': True}, 'pos'), ({'ramasuk': True, 'pakdhe': True, 'menjerat': True, 'perut': True}, 'pos'), ({'diperpanjang': True, 'kontrak': True, 'diputusin': True}, 'pos'), ({'mantap': True, 'berat': True}, 'pos'), ({'titip': True, 'jualan': True, 'mincot': True, 'kondisi': True, 'gini': True, 'kesulitan': True, 'ekonomi': True}, 'pos'), ({'masyarakat': True, 'menengah': True, 'kebawah': True}, 'pos'), ({'diperpanjang':